In [ ]:
giphy_api_key = ''

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [ ]:
import requests

def search_gifs(query, api_key, limit=10):
    """
    Search for GIFs based on text using the Giphy API.

    Parameters:
    - query: Text to search for.
    - api_key: Your Giphy API key.
    - limit: The maximum number of GIFs to return (default is 10).

    Returns:
    A list of URLs for the found GIFs.
    """
    base_url = "https://api.giphy.com/v1/gifs/search"
    params = {
        "api_key": api_key,
        "q": query,
        "limit": limit
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()['data']
        gif_urls = [gif['images']['original']['url'] for gif in data]
        return gif_urls
    else:
        print(f"Error: {response.status_code}")
        return []

# Example usage
query = "AI!!"  # Example search query
gif_urls = search_gifs(query, giphy_api_key)

for url in gif_urls:
    print(url)


In [ ]:
from IPython.display import Image as Img


In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageSequence
import requests
from io import BytesIO



In [ ]:
# Define your desired data structure.
from langchain_community.llms import Ollama
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

class Meme(BaseModel):
    text: str = Field(description="the rewritten meme")
    keywords: List = Field(description="the list of releveant search keywords")

        
class Memer:
    def __init__(self, local=False):
        
        if local:
            llm = Ollama(model="mistral:instruct", temperature=0.6)
        else:
            llm = ChatMistralAI(mistral_api_key='', model = 'mistral-small')
        parser = PydanticOutputParser(pydantic_object=Meme)

        prompt = PromptTemplate(
            template="Answer the user query.\n{format_instructions}\n{query}\n",
            input_variables=["query"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        


        chain = prompt | llm | parser
        self.chain = chain
    
    def get(self, text):
        c = 0
        while c < 5:
            try:
               
                ans = self.chain.invoke({"query": f'''You are a assistant who 
                should provide a funny shorter text about the given text and list of relevant memes to search memes on giphy when
                i provide a text {text}'''})
                
                return ans
            except:
                pass
            c+=1
            


In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageSequence
import requests
from io import BytesIO
import textwrap

cache = dict()

def create_gif_meme_with_text_enhanced(gif_url, text, font_path="Arial.ttf", font_size=20, wrap_width=40):
    """
    Create a GIF meme by adding wrapped text with a background box to each frame.

    Parameters:
    - gif_url: URL of the source GIF.
    - text: The text to add to the GIF.
    - font_path: Path to the font file.
    - font_size: Font size for the text.
    - wrap_width: Maximum line width before wrapping the text.
    """
    # Download the GIF
    if gif_url not  in cache:
        response = requests.get(gif_url)
        gif = Image.open(BytesIO(response.content))
        cache[gif_url] = gif
        
    gif = cache[gif_url]
    # Create font object for drawing text
    font = ImageFont.truetype(font_path, font_size)

    # Wrap the text
    wrapped_text = textwrap.fill(text.upper(), wrap_width)

    # Process each frame of the original GIF
    frames = []
    for frame in ImageSequence.Iterator(gif):
        rgba_frame = frame.convert("RGBA")
        draw = ImageDraw.Draw(rgba_frame)

        # Calculate text size and position
        text_width, text_height = draw.textsize(wrapped_text, font=font)
        text_x = (rgba_frame.width - text_width) // 2
        text_y = rgba_frame.height - text_height - 10

        # Draw semi-transparent rectangle as text background
        draw.rectangle(((text_x, text_y), (text_x + text_width, text_y + text_height)), fill=(0, 0, 0, 127))

        # Draw the wrapped text
        draw.multiline_text((text_x, text_y), wrapped_text, fill="white", font=font, align="center")
        
        frames.append(rgba_frame.copy())

    # Save the frames as a new GIF
    output_path = "enhanced_meme.gif"
    frames[0].save(output_path, save_all=True, append_images=frames[1:], optimize=False, loop=0, duration=gif.info['duration'])

    return output_path




In [ ]:
# Example usage (ensure you replace the placeholder values appropriately)
gif_url = gif_urls[4] # Replace with your actual GIF URL
text = "WALL-E"
output_path = create_gif_meme_with_text_enhanced(gif_url, text, font_size=20, wrap_width=30)
print(f"GIF meme created at {output_path}")

In [ ]:
Img(filename="enhanced_meme.gif")

In [ ]:
memer = Memer()

In [ ]:
meme = memer.get('open ai to be closed ai')

In [ ]:
text ='open ai to be closed ai'

for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[0]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, text)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)

In [ ]:
text ='open ai to be closed ai'

for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[1]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, text)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)

In [ ]:
text ='busy working!!!'
meme = memer.get(text)
for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[1]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, meme.text)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)

In [ ]:
text ='Wow!!'
meme = memer.get(text)
for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[0]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, text)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)

In [ ]:
for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[1]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, text, wrap_width=30)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)

In [ ]:
text ='Listen to Me Her Not A Real People'
meme = memer.get(text)

for t in meme.keywords:
    gif_urls = search_gifs(t, giphy_api_key)  # Make sure you have defined `search_gifs` function from previous examples

    # Example usage
    gif_url = gif_urls[0]  # Repla ce with your actual GIF URL

    output_path = create_gif_meme_with_text_enhanced(gif_url, text, wrap_width=30)
    gif = Img(filename="enhanced_meme.gif")
    display(gif)